<img src="../imgs/Adevinta-ULPGC-logo.jpg" width="530px" align="right">

# **Parameter-Efficient Fine-Tuning (PEFT)**

https://huggingface.co/docs/peft/index  <-- Empezar por aquí

El Parameter-Efficient Fine-Tuning (PEFT) se refiere a técnicas de ajuste fino que buscan mejorar o adaptar modelos de aprendizaje profundo preentrenados, especialmente modelos de lenguaje de gran escala, a tareas específicas sin necesidad de reentrenar completamente todos los parámetros del modelo. Este enfoque es particularmente valioso dada la creciente escala de los modelos de lenguaje natural (como GPT, BERT, y similares), cuyo entrenamiento completo requiere una cantidad significativa de recursos computacionales y energéticos.

PEFT aborda el desafío de cómo hacer que el ajuste fino de estos modelos sea más accesible y práctico, permitiendo personalizar los modelos a tareas específicas con menos recursos. Algunas de las técnicas dentro de este enfoque incluyen:

### 1. **Low-Rank Adaptation**

<a href="https://arxiv.org/pdf/2106.09685.pdf">LoRA: Low-Rank Adaptation of Large Language Models</a>

Esta técnica se basa en la adición de matrices de bajo rango que modifican las representaciones intermedias del modelo. Al ajustar estas matrices de bajo rango, se puede influir en el comportamiento del modelo con un costo computacional relativamente bajo.

<div align="center">
    <img src="./imgs/LoRA.png" width="350px">
</div>

Estas matrices de bajo rango se insertan en los parámetros de un modelo preentrenado, específicamente en los puntos de interacción clave como las matrices de transformación en capas de atención y capas feed-forward. 

Matemáticamente, esto se logra ya que la matriz de adaptación es el producto de dos matrices más pequeñas, facilitando así una modificación eficiente del comportamiento del modelo con una cantidad mínima de parámetros adicionales.

Consideremos una capa específica en un modelo, donde la operación lineal original es $ W $, una matriz que transforma la entrada $ x $ en la salida $ y $, es decir, $ y = Wx $.

1. **Introducción del Adaptador de Bajo Rango:**
   En LoRA, en lugar de modificar $ W $ directamente, se introduce una adaptación $ \Delta W $ que es de bajo rango. Matemáticamente, $ \Delta W $ se expresa como el producto de dos matrices de menor dimensión $ A $ y $ B $, es decir,
   $$
   \Delta W = AB
   $$
   donde $ A $ es una matriz de $ d \times r $ y $ B $ es una matriz de $ r \times d $. Aquí, $ r $ es el rango de la adaptación y es mucho menor que $ d $, la dimensión de la entrada y salida.

2. **Modificación de la Transformación Lineal:**
   La nueva transformación lineal del modelo con la adaptación de bajo rango se convierte en:
   $$
   y = (W + \Delta W)x = (W + AB)x
   $$
   Aquí, $ W $ es la matriz original y $ AB $ es la adaptación de bajo rango que ajusta la transformación de $ W $ para adaptarse mejor a una tarea específica.

3. **Cálculo de la Salida Modificada:**
   Al descomponer $ \Delta W $ en el producto de $ A $ y $ B $, reducimos el número de parámetros a entrenar desde $ d^2 $ (si se modificara $ W $ completamente) a $ d \times r + r \times d $, lo cual es significativamente menor si $ r $ es pequeño.

##### **Ejemplo en un Transformer**

Consideremos la capa de atención donde $ Q, K, V $ (consultas, claves y valores) son transformados por las matrices $ W^Q, W^K, W^V $. Con LoRA, se introducirían adaptaciones $ \Delta W^Q, \Delta W^K, \Delta W^V $ tal que:
   $$
   Q = (W^Q + \Delta W^Q)X, \quad K = (W^K + \Delta W^K)X, \quad V = (W^V + \Delta W^V)X
   $$
donde $ X $ es la entrada a la capa de atención y cada $ \Delta W $ es una adaptación de bajo rango específica para $ Q, K, $ o $ V $.

Este enfoque permite una flexibilidad considerable en la adaptación del modelo a nuevas tareas, permitiendo ajustes finos donde es más necesario con un mínimo impacto en el tamaño y la complejidad del modelo.


#### **QLoRA: Quantized Low-Rank Adaptation for Efficient Fine-Tuning of Pretrained Transformers**

QLoRA y LoRA son técnicas de adaptación para modelos de lenguaje de gran escala, pero QLoRA presenta cuantización, lo que significa que utiliza versiones de menor precisión de los números para reducir los requisitos de memoria y computación. Esto es especialmente útil para despliegues en dispositivos con recursos limitados.

### 2. **Adapters**

<a href="https://arxiv.org/pdf/1902.00751.pdf">Parameter-Efficient Transfer Learning for NLP</a>

Un "adapter" es esencialmente un módulo pequeño y entrenable que se inserta entre las capas preexistentes de un modelo de lenguaje grande. Estos módulos tienen una estructura específica, generalmente consistiendo en una capa de reducción de dimensionalidad, una transformación no lineal, y una capa de expansión de dimensionalidad. La idea es que estos adapters aprendan ajustes específicos para la tarea en cuestión, mientras que los parámetros originales del modelo (los de las grandes capas preentrenadas) permanecen congelados.

<div align="center">
    <img src="./imgs/adapters.webp" width="650px">
</div>

<div align="center">
    <img src="./imgs/adapters.png" width="250px">
</div>

### **Ejemplo: GPT2 - PEFT**


In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from torch.utils.data import Dataset, DataLoader
import torch

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/cayetano/Propio/Notebooks/Machine Learning/RL/env/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
class TextDataset(Dataset):
    def __init__(self, tokenizer, file_path, block_size=128):
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = [line.strip() for line in file if line.strip()]

        self.examples = []
        for line in lines:
            tokens = tokenizer.encode(line, add_special_tokens=True)
            if len(tokens) > block_size:
                tokens = tokens[:block_size]
            tokens += [tokenizer.eos_token_id]
            tokens += [tokenizer.pad_token_id] * (block_size - len(tokens))
            self.examples.append(torch.tensor(tokens))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        res = {"input_ids": self.examples[i][:-1], "labels": self.examples[i][1:]}
        # res = {"input_ids": self.examples[i]}
        return res

In [3]:
# Configuraciones
file_path = 'data/numbers_gpt.csv'  # Ajusta esto al path de tu archivo de texto
block_size = 128  # Longitud máxima de las secuencias de tokens

# Cargar tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token_id = tokenizer.eos_token_id

# Preparar el dataset
dataset = TextDataset(tokenizer, file_path, block_size)

# Crear DataLoader para iterar sobre el dataset durante el entrenamiento
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [14]:
model = AutoModelForCausalLM.from_pretrained('gpt2')


In [11]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [15]:

# Número de parámetros entrenables del modelo
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"El modelo tiene {num_params} parámetros entrenables")

# Número de parámetros del modelo
num_params = sum(p.numel() for p in model.parameters())
print(f"El modelo tiene {num_params} parámetros totales")

El modelo tiene 124439808 parámetros entrenables
El modelo tiene 124439808 parámetros totales


In [9]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
         

In [12]:
# Configurar LoRA
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj", "c_fc"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    fan_in_fan_out=True,
)
model = get_peft_model(model, config)

In [6]:
# Número de parámetros entrenables del modelo
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"El modelo tiene {num_params} parámetros entrenables")

# Número de parámetros del modelo
num_params = sum(p.numel() for p in model.parameters())
print(f"El modelo tiene {num_params} parámetros totales")

El modelo tiene 1179648 parámetros entrenables
El modelo tiene 125619456 parámetros


In [7]:
def custom_save_function(output_dir):
    model.save_pretrained(output_dir)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="modelo_entrenado",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="logs",
    logging_steps=10,
    save_steps=10,
    save_strategy="no",
)

# Crear el objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    # eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

# Entrenar el modelo
trainer.train()

# Guardar el modelo entrenado
model.save_pretrained("modelo_entrenado")
tokenizer.save_pretrained("modelo_entrenado")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenize

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/2500 [00:00<?, ?it/s]

{'loss': 12.5621, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 12.532, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 12.494, 'learning_rate': 3e-06, 'epoch': 0.01}
{'loss': 12.434, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 12.3959, 'learning_rate': 5e-06, 'epoch': 0.02}
{'loss': 12.2208, 'learning_rate': 6e-06, 'epoch': 0.02}
{'loss': 12.0261, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.03}
{'loss': 11.7139, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.03}
{'loss': 11.2695, 'learning_rate': 9e-06, 'epoch': 0.04}
{'loss': 10.8182, 'learning_rate': 1e-05, 'epoch': 0.04}
{'loss': 9.8953, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.04}
{'loss': 8.7168, 'learning_rate': 1.2e-05, 'epoch': 0.05}
{'loss': 7.2017, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.05}
{'loss': 5.7925, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.06}
{'loss': 4.4684, 'learning_rate': 1.5e-05, 'epoch': 0.06}
{'loss': 3.0273, 

('modelo_entrenado/tokenizer_config.json',
 'modelo_entrenado/special_tokens_map.json',
 'modelo_entrenado/vocab.json',
 'modelo_entrenado/merges.txt',
 'modelo_entrenado/added_tokens.json',
 'modelo_entrenado/tokenizer.json')

#### **Generación del texto**

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Cargar el modelo y el tokenizador entrenados
model_path = "modelo_entrenado"
tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(model_path)

# Generar número en formato textual
def generate_textual_number(numeric_number):
    input_text = str(numeric_number)
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to("mps")
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    textual_number = tokenizer.decode(output[0], skip_special_tokens=True)
    return textual_number

# Ejemplo de uso
numeric_number = 123456789
textual_number = generate_textual_number(numeric_number)
print(f"El número {numeric_number} en formato textual es: {textual_number}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


El número 123456789 en formato textual es: 123456789, hundred million hundred nine five eight fifty thousand hundred nine


wandb: Network error (ConnectTimeout), entering retry loop.
